<a href="https://colab.research.google.com/github/nanhaishun/leaf_practice/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/albumentations-team/albumentations.git

  Cloning https://github.com/albumentations-team/albumentations.git to /tmp/pip-req-build-xac_yrfc
  Running command git clone -q https://github.com/albumentations-team/albumentations.git /tmp/pip-req-build-xac_yrfc
     |████████████████████████████████| 952kB 6.8MB/s 
  Created wheel for albumentations: filename=albumentations-0.5.2-cp36-none-any.whl size=81845 sha256=6b11a8bacb578d37c56a06cb1d438b89589b9901f19652c96a77773fbb2c1c4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-hj6kxq3k/wheels/e2/85/3e/2a40fac5cc1f43ced656603bb2fca1327b30ec7de1b1b66517
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [7]:
!pip install timm

In [8]:
!git clone https://github.com/nanhaishun/leaf_practice.git

Cloning into 'leaf_practice'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [5]:
import os
os.getcwd()

'/content'

In [6]:
# 导入依赖的库
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import StratifiedKFold
from torch.cuda.amp import GradScaler
from torch import nn
from tqdm import tqdm
import torch
import timm
import cv2
import pandas as pd
import numpy as np
from utils import utils
from imp import reload
from albumentations.pytorch import ToTensorV2
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout,
    ShiftScaleRotate, CenterCrop, Resize
)
reload(utils)
rand_seed = 666
utils.seed_everything(rand_seed)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!pip install kaggle

从kaggle下载数据集

In [17]:
!mkdir -p ~/.kaggle 
!cp /content/kaggle.json ~/.kaggle/ 
!chmod 600 ~/.kaggle/kaggle.json 
#kaggle competitions download -c cassava-leaf-disease-classification
!kaggle competitions download -c cassava-leaf-disease-classification -p '/content/data/'

  0% 0.00/141k [00:00<?, ?B/s]
100% 141k/141k [00:00<00:00, 55.6MB/s]
  0% 0.00/191k [00:00<?, ?B/s]
100% 191k/191k [00:00<00:00, 59.1MB/s]
  0% 0.00/135k [00:00<?, ?B/s]
100% 135k/135k [00:00<00:00, 51.8MB/s]
  0% 0.00/101k [00:00<?, ?B/s]
100% 101k/101k [00:00<00:00, 89.1MB/s]
  0% 0.00/110k [00:00<?, ?B/s]
100% 110k/110k [00:00<00:00, 113MB/s]
  0% 0.00/101k [00:00<?, ?B/s]
100% 101k/101k [00:00<00:00, 83.2MB/s]
  0% 0.00/72.8k [00:00<?, ?B/s]
100% 72.8k/72.8k [00:00<00:00, 61.7MB/s]
  0% 0.00/104k [00:00<?, ?B/s]
100% 104k/104k [00:00<00:00, 105MB/s]
  0% 0.00/82.3k [00:00<?, ?B/s]
100% 82.3k/82.3k [00:00<00:00, 69.2MB/s]
  0% 0.00/127k [00:00<?, ?B/s]
100% 127k/127k [00:00<00:00, 37.9MB/s]
  0% 0.00/170k [00:00<?, ?B/s]
100% 170k/170k [00:00<00:00, 52.6MB/s]
  0% 0.00/79.1k [00:00<?, ?B/s]
100% 79.1k/79.1k [00:00<00:00, 68.7MB/s]
  0% 0.00/131k [00:00<?, ?B/s]
100% 131k/131k [00:00<00:00, 41.3MB/s]
  0% 0.00/136k [00:00<?, ?B/s]
100% 136k/136k [00:00<00:00, 118MB/s]
  0% 0.00/158k

In [11]:
train_img_path = r'../data/train_images' #样本图片的路径
train_csv_path = r'../data/train.csv' #训练集合标记CSV

In [ ]:
# 训练集数据增强
def get_train_transforms():
    return Compose([
        RandomResizedCrop(CFG['img_size'], CFG['img_size']),
        Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        ShiftScaleRotate(p=0.5),
        HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        CoarseDropout(p=0.5),
        Cutout(p=0.5),
        ToTensorV2(p=1.0),
    ], p=1.)

# 验证集数据增强
def get_valid_transforms():
    return Compose([
        CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
        Resize(CFG['img_size'], CFG['img_size']),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)

In [ ]:
# 模型构建
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
# 构建数据
CFG = {
    'img_size' : 512,
    'epochs': 15,
    'fold_num': 5,
    'device': 'cpu',
    'model_arch': 'tf_efficientnet_b4_ns',
    'train_bs' : 2,
    'valid_bs' : 2,
    'num_workers' : 0,
    'lr': 1e-4,
    'weight_decay': 1e-6,
    'T_0': 10,
    'min_lr': 1e-6,
}
train = pd.read_csv(train_csv_path)
folds = StratifiedKFold(n_splits=CFG['fold_num'],
                        shuffle=True,
                        random_state=rand_seed).split(
                            np.arange(train.shape[0]), train.label.values)
trn_transform = get_train_transforms()
val_transform = get_valid_transforms()

In [ ]:
fold_num = 0
for fold, (trn_idx, val_idx) in enumerate(folds):
    if fold == fold_num:
        print('Training with {} started'.format(fold))
        print('Train : {}, Val : {}'.format(len(trn_idx), len(val_idx)))
        train_loader, val_loader = utils.prepare_dataloader(train,
                                                          trn_idx,
                                                          val_idx,
                                                          data_root = train_img_path,
                                                          trn_transform = trn_transform,
                                                          val_transform = val_transform, 
                                                          bs = CFG['train_bs'], 
                                                          n_job = CFG['num_workers'])

        device = torch.device(CFG['device'])

        model = CassvaImgClassifier(CFG['model_arch'],
                                    train.label.nunique(),
                                    pretrained=True).to(device)
        scaler = GradScaler()
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=CFG['lr'],
                                     weight_decay=CFG['weight_decay'])

        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer,
            T_0=CFG['T_0'],
            T_mult=1,
            eta_min=CFG['min_lr'],
            last_epoch=-1)

        loss_tr = nn.CrossEntropyLoss().to(
            device)
        loss_fn = nn.CrossEntropyLoss().to(device)

        for epoch in range(CFG['epochs']):
            utils.train_one_epoch(epoch,
                                model,
                                loss_tr,
                                optimizer,
                                train_loader,
                                device,
                                scaler,
                                scheduler=scheduler,
                                schd_batch_update=False)

            with torch.no_grad():
                utils.valid_one_epoch(epoch,
                                    model,
                                    loss_fn,
                                    val_loader,
                                    device)

            torch.save(
                model.state_dict(),
                '../model/{}_fold_{}_{}'.format(CFG['model_arch'], fold, epoch))

        del model, optimizer, train_loader, val_loader, scaler, scheduler
        torch.cuda.empty_cache()


Training with 0 started
Train : 17117, Val : 4280


/Users/wanjun/anaconda/envs/python36/lib/python3.6/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")



  0%|          | 0/8559 [00:00<?, ?it/s]/Users/wanjun/anaconda/envs/python36/lib/python3.6/site-packages/torch/cuda/amp/autocast_mode.py:114: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")



epoch 0 loss: 1.6059:   0%|          | 0/8559 [00:07<?, ?it/s]


epoch 0 loss: 1.6059:   0%|          | 1/8559 [00:07<18:46:41,  7.90s/it]


epoch 0 loss: 1.6060:   0%|          | 1/8559 [00:14<18:46:41,  7.90s/it]


epoch 0 loss: 1.6060:   0%|          | 2/8559 [00:14<17:35:07,  7.40s/it]

KeyboardInterrupt: 